In [1]:
import pandas as pd
from keras.callbacks import History, ReduceLROnPlateau,EarlyStopping,ModelCheckpoint
import os
import numpy as np
from data_analysis import calculate_metrics, load_weights_and_evaluate
from model_builders import GCN_pretraining
from hyperparameter_tuning_GCN import objective
from functools import partial
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
import pickle
import dill

Using TensorFlow backend.
C:\Users\tomas\miniconda3\envs\binding\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\tomas\miniconda3\envs\binding\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\tomas\miniconda3\envs\binding\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\tomas\miniconda3\envs\binding

In [ ]:
# Specify model callbacks on training
es = EarlyStopping(monitor='loss',patience=8, min_delta=0)
rlr = ReduceLROnPlateau(monitor='loss',factor=0.5, patience=4, verbose=1, min_lr=0.0000001)

model_params = {
        "num_layers" : 3,
        "max_atoms" : 70,
        "num_atom_features" : 62,
        "num_atom_features_original" : 62,
        "num_bond_features" : 6,
        "max_degree" : 5,
        "conv_width" : [32,64,96],
        "fp_length" : [96,96,96],
        "activ_enc" : "selu",
        "activ_dec" : "selu",
        "learning_rates" : [0.001,0.001,0.001],
        "learning_rates_fp": [0.005,0.005,0.005],
        "losses_conv" : {
                    "neighbor_output": "mean_squared_error",
                    "self_output": "mean_squared_error",
                    },
        "lossWeights" : {"neighbor_output": 1.0, "self_output": 1.0},
        "metrics" : "mse",
        "loss_fp" : "mean_squared_error",
        "enc_layer_names" : ["enc_1", "enc_2", "enc_3"],
        'callbacks' : [es,rlr],
        'adam_decay': 0.0005329142291371636,
        'beta': 5,
        'p': 0.004465204118126482,
        'dense_size' : [96,96,48],
        'dropout_rate' : [0.1,0.1],
        'lr' : 0.001,
        'batch_size' : int(64),
        'n_epochs' : int(5)
        }


In [2]:
fspace = {
    'conv1' : hp.quniform('conv1', 32, 96, 8),
    'conv2' : hp.quniform('conv2', 48, 128, 8),
    'conv3' : hp.quniform('conv3', 64, 168, 8),
    'fp' : hp.quniform('fp', 64, 196, 8),
    'dense1' : hp.quniform('dense1',96,256,32),
    'dense2' : hp.quniform('dense2',96,256,32),
    'dense3' : hp.quniform('dense3',48,128,32),
    'dropout_rate' : hp.uniform('dropout_rate',0.1,0.5),
    'lr' : hp.uniform('lr',0.0005,0.01),
    'n_epochs' : hp.quniform('n_epochs',15,40,5) 
}

In [4]:
target_1 = 'p38'
base_path_1 = f'C:/Users/tomas/Documents/GitHub/kinase_binding'

data_fpath_1 = base_path_1+f'/data/{target_1}/data.csv'
df_p38=pd.read_csv(data_fpath_1).set_index('biolab_index')

with open(base_path_1+f'/data/{target_1}/train_val_folds.pkl', "rb") as in_f:
    train_val_folds_p38 = dill.load(in_f)

with open(base_path_1+f'/data/{target_1}/train_test_folds.pkl', "rb") as in_f:
    train_test_folds_p38 = dill.load(in_f)

training_p38 = [df_p38.loc[train_val_folds_p38[0][0]],
                 df_p38.loc[train_val_folds_p38[1][0]],
                 df_p38.loc[train_val_folds_p38[2][0]],
                 df_p38.loc[train_val_folds_p38[3][0]],
                 df_p38.loc[train_val_folds_p38[4][0]],
                 df_p38.loc[train_val_folds_p38[5][0]],
                 ]
validation_p38 = [df_p38.loc[train_val_folds_p38[0][1]],
                   df_p38.loc[train_val_folds_p38[1][1]],
                   df_p38.loc[train_val_folds_p38[2][1]],
                   df_p38.loc[train_val_folds_p38[3][1]],
                   df_p38.loc[train_val_folds_p38[4][1]],
                   df_p38.loc[train_val_folds_p38[5][1]],
                   ]

In [ ]:
# no need for manual changes in this cell
train_files = os.path.join(base_path, f'data/{target}/split_aveb/fold_{{}}/train_{{}}.csv')
val_files = os.path.join(base_path, f'data/{target}/split_aveb/fold_{{}}/val_{{}}.csv')
weight_files = os.path.join(base_path, f'results/{target}/{model_name}/fold_{{}}/model_weights/model_{{}}.h5')

In [ ]:
train_sets = [pd.read_csv(train_files.format(i,i), index_col = 0) if 'Unnamed: 0' in pd.read_csv(train_files.format(i,i)) else pd.read_csv(train_files.format(i,i)) for i in range(7)]
val_sets = [pd.read_csv(val_files.format(i,i), index_col = 0) if 'Unnamed: 0' in pd.read_csv(val_files.format(i,i)) else pd.read_csv(val_files.format(i,i)) for i in range(7)]
test_set = pd.read_csv(os.path.join(base_path, f'data/{target}/split_aveb/test.csv')) 

In [5]:
fmin_objective = partial(objective, train_sets = training_p38, val_sets = validation_p38)

In [8]:
def run_trials():

    trials_step = 0  # how many additional trials to do after loading saved trials. 1 = save after iteration
    max_trials = 1  # initial max_trials. put something small to not have to wait

    
    try:  # try to load an already saved trials object, and increase the max
        trials = pickle.load(open("gcn.hyperopt", "rb"))
        print("Found saved Trials! Loading...")
        max_trials = len(trials.trials) + trials_step
        print("Rerunning from {} trials to {} (+{}) trials".format(len(trials.trials), max_trials, trials_step))
    except:  # create a new trials object and start searching
        trials = Trials()

    best = fmin(fn = fmin_objective, space = fspace, algo=tpe.suggest, max_evals=max_trials, trials=trials)

    print("Best:", best)
    
    # save the trials object
    with open("gcn.hyperopt", "wb") as f:
        pickle.dump(trials, f)
    return(trials)

In [9]:
trials = run_trials()

Found saved Trials! Loading...
Rerunning from 555 trials to 555 (+0) trials
100%|████████████████████████████████████████████████████████████| 555/555 [00:00<00:00, 547083.13trial/s, best loss=?]
Best: {'conv1': 96.0, 'conv2': 104.0, 'conv3': 120.0, 'dense1': 256.0, 'dense2': 192.0, 'dense3': 96.0, 'dropout_rate': 0.3554537312557061, 'fp': 160.0, 'lr': 0.007037117031430456, 'n_epochs': 35.0}


In [ ]:
trials.trials[92]

In [ ]:
run_trials()